# 🚀 NASA Defect Prediction: CM1

**Dataset:** CM1
**Method:** Baseline RF → KAN Base → KAN + Attention
**Goal:** F2 & Recall optimization (defect detection)

**✅ Self-contained:** Run all cells in order, no dependencies!

---

## 📦 Step 1: Setup & Environment

In [ ]:
# Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('✅ Google Drive mounted!')
except ImportError:
    print('⚠️  Not on Colab - skipping mount')


In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install imbalanced-learn scipy scikit-learn torch matplotlib seaborn pandas numpy openpyxl -q
print('✅ Packages installed!')


In [ ]:
# Import libraries
import os
import json
import warnings
import datetime
import numpy as np
import pandas as pd
from scipy.io import arff
from io import StringIO

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    fbeta_score, confusion_matrix, average_precision_score
)
from imblearn.over_sampling import SMOTE

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

warnings.filterwarnings('ignore')
print('✅ Imports complete!')


In [ ]:
# Configuration
DATASET_NAME = 'CM1'
DATASET_PATH = '/content/drive/MyDrive/nasa-defect-gwo-kan/dataset'
OUTPUT_DIR = f'./results_{DATASET_NAME}'
SEED = 42

# Set seeds for reproducibility
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device('cpu')
os.makedirs(OUTPUT_DIR, exist_ok=True)

RUN_ID = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

print(f'✅ Configuration complete!')
print(f'📊 Dataset: CM1')
print(f'🖥️  Device: {device}')
print(f'📁 Output: {OUTPUT_DIR}')


## 🛠️ Step 2: Define Utility Functions

In [ ]:
# Utility functions for data loading and metrics

def load_arff(file_path):
    """Load ARFF file and return pandas DataFrame."""
    try:
        data, _ = arff.loadarff(file_path)
        df = pd.DataFrame(data)
        for col in df.columns:
            if df[col].dtype == object:
                try:
                    df[col] = df[col].str.decode('utf-8')
                except:
                    pass
        return df
    except:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()
        data_start = content.lower().find('@data')
        data_section = content[data_start + 5:].strip()
        return pd.read_csv(StringIO(data_section), header=None)

def calc_metrics(y_true, y_pred, y_proba=None):
    """Calculate comprehensive metrics for defect prediction."""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    m = {
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'f2': fbeta_score(y_true, y_pred, beta=2, zero_division=0),
        'accuracy': accuracy_score(y_true, y_pred),
        'tp': int(tp), 'fp': int(fp), 'tn': int(tn), 'fn': int(fn)
    }
    if y_proba is not None:
        try:
            m['pr_auc'] = average_precision_score(y_true, y_proba)
        except:
            m['pr_auc'] = 0.0
    else:
        m['pr_auc'] = 0.0
    return m

def find_threshold(y_true, y_proba):
    """Find optimal threshold for F2 score."""
    best_score, best_t = -1, 0.5
    for t in np.arange(0.05, 0.96, 0.05):
        y_pred = (y_proba >= t).astype(int)
        m = calc_metrics(y_true, y_pred)
        score = m['f2'] if m['accuracy'] >= 0.5 else 0
        if score > best_score:
            best_score, best_t = score, t
    return best_t

print('✅ Utility functions defined!')


## 🧠 Step 3: Define KAN Model Architecture

In [ ]:
# KANLinear: Core KAN layer with spline-based activation

class KANLinear(nn.Module):
    """Kolmogorov-Arnold Network Linear Layer with learnable spline activations."""
    
    def __init__(self, in_features, out_features, grid_size=3, spline_order=2):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order
        
        # Learnable grid points
        self.grid = nn.Parameter(
            torch.linspace(-1, 1, grid_size)
            .unsqueeze(0).unsqueeze(0)
            .repeat(out_features, in_features, 1)
        )
        
        # Spline coefficients
        self.coef = nn.Parameter(
            torch.randn(out_features, in_features, grid_size + spline_order) * 0.1
        )
        
        # Base weights
        self.base_weight = nn.Parameter(
            torch.randn(out_features, in_features) * 0.1
        )
    
    def forward(self, x):
        batch_size = x.shape[0]
        
        # Expand input for broadcasting
        x_expanded = x.unsqueeze(1).unsqueeze(-1)  # [B, 1, in_f, 1]
        
        # Compute distances to grid points
        distances = torch.abs(x_expanded - self.grid.unsqueeze(0))
        
        # Build basis functions
        basis = torch.zeros(
            batch_size, self.out_features, self.in_features,
            self.grid_size + self.spline_order,
            device=x.device
        )
        
        # RBF basis for grid points
        for i in range(self.grid_size):
            basis[:, :, :, i] = torch.exp(-distances[:, :, :, i] ** 2 / 0.5)
        
        # Polynomial basis
        for i in range(self.spline_order):
            basis[:, :, :, self.grid_size + i] = x_expanded.squeeze(-1) ** (i + 1)
        
        # Compute spline output
        spline_output = (basis * self.coef.unsqueeze(0)).sum(dim=-1).sum(dim=-1)
        
        # Add base transformation
        base_output = torch.matmul(x, self.base_weight.t())
        
        return spline_output + base_output

print('✅ KANLinear layer defined!')


In [ ]:
# KAN: Complete KAN model for defect prediction

class KAN(nn.Module):
    """Kolmogorov-Arnold Network for binary defect classification."""
    
    def __init__(self, input_dim, hidden_dim=64, grid_size=5, spline_order=3):
        super().__init__()
        # Deeper architecture with 3 KAN layers
        self.kan1 = KANLinear(input_dim, hidden_dim, grid_size, spline_order)
        self.kan2 = KANLinear(hidden_dim, hidden_dim, grid_size, spline_order)
        self.kan3 = KANLinear(hidden_dim, hidden_dim // 2, grid_size, spline_order)
        self.output_layer = nn.Linear(hidden_dim // 2, 1)
        
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim // 2)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.kan1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.kan2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.kan3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.output_layer(x)
        return torch.sigmoid(x)

print('✅ KAN model defined!')

In [ ]:
# Multi-Head Attention and Enhanced KAN with Attention

class MultiHeadAttention(nn.Module):
    """Multi-head attention mechanism for feature importance."""
    
    def __init__(self, input_dim, num_heads=4, attention_dim=32):
        super().__init__()
        self.num_heads = num_heads
        self.attention_dim = attention_dim
        
        # Multiple attention heads
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, attention_dim),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(attention_dim, input_dim),
                nn.Sigmoid()
            ) for _ in range(num_heads)
        ])
        
        # Combine heads
        self.combine = nn.Linear(input_dim * num_heads, input_dim)
        self.layer_norm = nn.LayerNorm(input_dim)
    
    def forward(self, x):
        # Apply all attention heads
        attended_outputs = []
        for head in self.heads:
            attention_weights = head(x)
            attended_outputs.append(x * attention_weights)
        
        # Concatenate and combine
        combined = torch.cat(attended_outputs, dim=-1)
        output = self.combine(combined)
        
        # Residual connection + layer norm
        output = self.layer_norm(x + output)
        
        return output


class KAN_Attention(nn.Module):
    """Enhanced KAN with Multi-Head Attention for superior defect detection."""
    
    def __init__(self, input_dim, hidden_dim=128, grid_size=5, spline_order=3):
        super().__init__()
        
        # Multi-head attention with 4 heads
        self.attention = MultiHeadAttention(input_dim, num_heads=4, attention_dim=64)
        
        # Deeper KAN architecture - 4 layers
        self.kan1 = KANLinear(input_dim, hidden_dim, grid_size, spline_order)
        self.kan2 = KANLinear(hidden_dim, hidden_dim, grid_size, spline_order)
        self.kan3 = KANLinear(hidden_dim, hidden_dim // 2, grid_size, spline_order)
        self.kan4 = KANLinear(hidden_dim // 2, hidden_dim // 4, grid_size, spline_order)
        
        # Batch normalization for each layer
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.bn3 = nn.BatchNorm1d(hidden_dim // 2)
        self.bn4 = nn.BatchNorm1d(hidden_dim // 4)
        
        # Dropout for regularization
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.15)
        self.dropout4 = nn.Dropout(0.1)
        
        # Output layer
        self.output_layer = nn.Linear(hidden_dim // 4, 1)
        
        # Feature importance tracking
        self.feature_importance = None
    
    def forward(self, x):
        # Apply multi-head attention with residual
        x_attended = self.attention(x)
        
        # Deep KAN layers with residual connections
        x1 = self.kan1(x_attended)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)
        x1 = self.dropout1(x1)
        
        x2 = self.kan2(x1)
        x2 = self.bn2(x2)
        x2 = F.relu(x2)
        x2 = self.dropout2(x2)
        
        # Residual connection from x1 to x2
        x2 = x2 + x1
        
        x3 = self.kan3(x2)
        x3 = self.bn3(x3)
        x3 = F.relu(x3)
        x3 = self.dropout3(x3)
        
        x4 = self.kan4(x3)
        x4 = self.bn4(x4)
        x4 = F.relu(x4)
        x4 = self.dropout4(x4)
        
        # Final prediction
        output = self.output_layer(x4)
        return torch.sigmoid(output)

print('✅ Multi-Head Attention and Enhanced KAN_Attention defined!')

In [ ]:
# Focal Loss for imbalanced classification

class FocalLoss(nn.Module):
    """Focal Loss to handle class imbalance - optimized for minority class."""
    
    def __init__(self, alpha=0.75, gamma=3.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, inputs, targets):
        bce_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal_loss.mean()

print('✅ Optimized Focal Loss defined!')

## 📊 Step 4: Load and Preprocess Data

In [ ]:
# Load dataset
print('\n' + '='*70)
print('🚀 LOADING DATASET: CM1')
print('='*70 + '\n')

file_path = os.path.join(DATASET_PATH, 'CM1.arff')
df = load_arff(file_path)

# Separate features and labels
X = df.iloc[:, :-1].values.astype(np.float32)
y = df.iloc[:, -1].values

# Encode labels if needed
if y.dtype == object:
    y = LabelEncoder().fit_transform(y)
else:
    y = y.astype(int)

print(f'✅ Dataset loaded successfully!')
print(f'   Total samples: {len(y)}')
print(f'   Features: {X.shape[1]}')
print(f'   Defective samples: {np.sum(y==1)} ({np.mean(y==1):.2%})')
print(f'   Non-defective samples: {np.sum(y==0)} ({np.mean(y==0):.2%})')


In [ ]:
# Handle missing values (NaN imputation)
if np.any(np.isnan(X)):
    print('⚠️  Found NaN values, imputing with column medians...')
    col_medians = np.nanmedian(X, axis=0)
    nan_indices = np.where(np.isnan(X))
    X[nan_indices] = np.take(col_medians, nan_indices[1])
    print('✅ NaN values imputed!')
else:
    print('✅ No NaN values found!')


In [ ]:
# Split data (train/val/test) - leakage-free splits
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=SEED
)

print('✅ Data split complete!')
print(f'   Training samples: {len(y_train)}')
print(f'   Validation samples: {len(y_val)}')
print(f'   Test samples: {len(y_test)}')


In [ ]:
# Feature scaling (fit on train, transform all sets)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print('✅ Feature scaling complete!')


In [ ]:
# Apply SMOTE to training data only (prevent data leakage)
smote = SMOTE(sampling_strategy=1.0, random_state=SEED)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print('✅ SMOTE resampling complete!')
print(f'   Before: {len(y_train)} samples')
print(f'   After: {len(y_train_resampled)} samples')
print(f'   Added: {len(y_train_resampled) - len(y_train)} synthetic samples')
print(f'   Class balance: {np.sum(y_train_resampled==1)}/{np.sum(y_train_resampled==0)} (defective/normal)')

# Initialize results dictionary
results = {}

## 🌲 Step 5: Train Baseline Random Forest

In [ ]:
# Train baseline Random Forest classifier
print('\n' + '='*70)
print('🌲 TRAINING BASELINE: RANDOM FOREST')
print('='*70 + '\n')

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1
)

rf_model.fit(X_train_resampled, y_train_resampled)
print('✅ Random Forest training complete!')

# Find optimal threshold on validation set
y_val_proba_rf = rf_model.predict_proba(X_val)[:, 1]
threshold_rf = find_threshold(y_val, y_val_proba_rf)
print(f'   Optimal threshold: {threshold_rf:.2f}')

# Evaluate on test set
y_test_proba_rf = rf_model.predict_proba(X_test)[:, 1]
y_test_pred_rf = (y_test_proba_rf >= threshold_rf).astype(int)
metrics_rf = calc_metrics(y_test, y_test_pred_rf, y_test_proba_rf)

print(f'\n📊 Test Set Results:')
print(f'   Recall:    {metrics_rf["recall"]:.4f}')
print(f'   Precision: {metrics_rf["precision"]:.4f}')
print(f'   F1 Score:  {metrics_rf["f1"]:.4f}')
print(f'   F2 Score:  {metrics_rf["f2"]:.4f}')
print(f'   Accuracy:  {metrics_rf["accuracy"]:.4f}')
print(f'   PR-AUC:    {metrics_rf["pr_auc"]:.4f}')

results['Baseline_RF'] = {'metrics': metrics_rf, 'threshold': threshold_rf}


## 🔥 Step 6: Train KAN Base Model

In [ ]:
# Train KAN base model
print('\n' + '='*70)
print('🔥 TRAINING KAN BASE MODEL')
print('='*70 + '\n')

# Initialize model
model_kan = KAN(
    input_dim=X.shape[1],
    hidden_dim=64,
    grid_size=5,
    spline_order=3
).to(device)

optimizer = optim.AdamW(model_kan.parameters(), lr=0.01, weight_decay=1e-4)
criterion = FocalLoss(alpha=0.75, gamma=3.0)

# Learning rate scheduler with warmup
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-5)

# Prepare data loaders
X_train_tensor = torch.FloatTensor(X_train_resampled).to(device)
y_train_tensor = torch.FloatTensor(y_train_resampled).unsqueeze(1).to(device)
X_val_tensor = torch.FloatTensor(X_val).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop with early stopping
best_val_f2 = 0
patience_counter = 0
max_patience = 20

for epoch in range(100):
    model_kan.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model_kan(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    scheduler.step()
    
    # Validation
    model_kan.eval()
    with torch.no_grad():
        val_outputs = model_kan(X_val_tensor).cpu().numpy().flatten()
        val_predictions = (val_outputs >= 0.5).astype(int)
        val_f2 = fbeta_score(y_val, val_predictions, beta=2, zero_division=0)
    
    if val_f2 > best_val_f2:
        best_val_f2 = val_f2
        best_model_state = model_kan.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
    
    if (epoch + 1) % 10 == 0:
        print(f'   Epoch {epoch + 1}/100 - Loss: {epoch_loss/len(train_loader):.4f} - Val F2: {val_f2:.4f} - LR: {scheduler.get_last_lr()[0]:.6f}')
    
    if patience_counter >= max_patience:
        print(f'   Early stopping at epoch {epoch + 1}')
        model_kan.load_state_dict(best_model_state)
        break

print(f'✅ KAN training complete!')
print(f'   Best validation F2: {best_val_f2:.4f}')

# Evaluate on test set
model_kan.eval()
with torch.no_grad():
    y_val_proba_kan = model_kan(X_val_tensor).cpu().numpy().flatten()
    X_test_tensor = torch.FloatTensor(X_test).to(device)
    y_test_proba_kan = model_kan(X_test_tensor).cpu().numpy().flatten()

threshold_kan = find_threshold(y_val, y_val_proba_kan)
print(f'   Optimal threshold: {threshold_kan:.2f}')

y_test_pred_kan = (y_test_proba_kan >= threshold_kan).astype(int)
metrics_kan = calc_metrics(y_test, y_test_pred_kan, y_test_proba_kan)

print(f'\n📊 Test Set Results:')
print(f'   Recall:    {metrics_kan["recall"]:.4f}')
print(f'   Precision: {metrics_kan["precision"]:.4f}')
print(f'   F1 Score:  {metrics_kan["f1"]:.4f}')
print(f'   F2 Score:  {metrics_kan["f2"]:.4f}')
print(f'   Accuracy:  {metrics_kan["accuracy"]:.4f}')
print(f'   PR-AUC:    {metrics_kan["pr_auc"]:.4f}')

results['KAN_Base'] = {'metrics': metrics_kan, 'threshold': threshold_kan}

## 🌟 Step 7: Train KAN + Attention Model

In [ ]:
# Train KAN with Attention mechanism - ENHANCED VERSION
print('\n' + '='*70)
print('🌟 TRAINING ENHANCED KAN + MULTI-HEAD ATTENTION MODEL')
print('='*70 + '\n')

# Initialize enhanced model with larger capacity
model_kan_att = KAN_Attention(
    input_dim=X.shape[1],
    hidden_dim=128,  # Much larger capacity
    grid_size=5,
    spline_order=3
).to(device)

print(f'📊 Model Parameters: {sum(p.numel() for p in model_kan_att.parameters()):,}')

# Optimizer with lower learning rate for better convergence
optimizer = optim.AdamW(model_kan_att.parameters(), lr=0.005, weight_decay=1e-3)

# Stronger focal loss for minority class
criterion = FocalLoss(alpha=0.85, gamma=4.0)

# Warmup + Cosine Annealing scheduler
def get_lr(epoch, warmup_epochs=5, total_epochs=150):
    if epoch < warmup_epochs:
        return 0.005 * (epoch + 1) / warmup_epochs
    else:
        progress = (epoch - warmup_epochs) / (total_epochs - warmup_epochs)
        return 1e-5 + (0.005 - 1e-5) * 0.5 * (1 + np.cos(np.pi * progress))

# Training loop with advanced techniques
best_val_f2 = 0
best_val_recall = 0
patience_counter = 0
max_patience = 30

training_history = {'loss': [], 'val_f2': [], 'val_recall': []}

for epoch in range(150):
    model_kan_att.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        # Update learning rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = get_lr(epoch)
        
        optimizer.zero_grad()
        outputs = model_kan_att(batch_X)
        loss = criterion(outputs, batch_y)
        
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model_kan_att.parameters(), max_norm=1.0)
        
        optimizer.step()
        epoch_loss += loss.item()
    
    # Validation
    model_kan_att.eval()
    with torch.no_grad():
        val_outputs = model_kan_att(X_val_tensor).cpu().numpy().flatten()
        val_predictions = (val_outputs >= 0.5).astype(int)
        val_f2 = fbeta_score(y_val, val_predictions, beta=2, zero_division=0)
        val_recall = recall_score(y_val, val_predictions, zero_division=0)
    
    training_history['loss'].append(epoch_loss / len(train_loader))
    training_history['val_f2'].append(val_f2)
    training_history['val_recall'].append(val_recall)
    
    # Save best model based on F2 score
    if val_f2 > best_val_f2:
        best_val_f2 = val_f2
        best_val_recall = val_recall
        best_model_state = model_kan_att.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
    
    if (epoch + 1) % 10 == 0:
        current_lr = get_lr(epoch)
        print(f'   Epoch {epoch + 1}/150 - Loss: {epoch_loss/len(train_loader):.4f} - Val F2: {val_f2:.4f} - Val Recall: {val_recall:.4f} - LR: {current_lr:.6f}')
    
    if patience_counter >= max_patience:
        print(f'   Early stopping at epoch {epoch + 1}')
        model_kan_att.load_state_dict(best_model_state)
        break

print(f'\n✅ KAN + Attention training complete!')
print(f'   Best validation F2: {best_val_f2:.4f}')
print(f'   Best validation Recall: {best_val_recall:.4f}')

# Evaluate on test set with aggressive recall-focused threshold optimization
model_kan_att.eval()
with torch.no_grad():
    y_val_proba_att = model_kan_att(X_val_tensor).cpu().numpy().flatten()
    y_test_proba_att = model_kan_att(X_test_tensor).cpu().numpy().flatten()

# Custom threshold optimization focusing on RECALL
print(f'\n🎯 Finding optimal threshold (recall-focused)...')
best_score = -1
best_threshold = 0.5
best_metrics = None

for t in np.arange(0.05, 0.80, 0.02):  # Lower thresholds for better recall
    y_val_pred_temp = (y_val_proba_att >= t).astype(int)
    temp_metrics = calc_metrics(y_val, y_val_pred_temp)
    
    # Weighted score: 60% recall + 40% F2
    score = 0.6 * temp_metrics['recall'] + 0.4 * temp_metrics['f2']
    
    # Must maintain minimum accuracy
    if temp_metrics['accuracy'] >= 0.55 and score > best_score:
        best_score = score
        best_threshold = t
        best_metrics = temp_metrics

threshold_att = best_threshold
print(f'   Optimal threshold: {threshold_att:.3f}')
print(f'   At this threshold - Val Recall: {best_metrics["recall"]:.4f}, Val F2: {best_metrics["f2"]:.4f}')

y_test_pred_att = (y_test_proba_att >= threshold_att).astype(int)
metrics_att = calc_metrics(y_test, y_test_pred_att, y_test_proba_att)

print(f'\n📊 Test Set Results:')
print(f'   Recall:    {metrics_att["recall"]:.4f} {"🎯 EXCELLENT!" if metrics_att["recall"] >= 0.85 else ""}')
print(f'   Precision: {metrics_att["precision"]:.4f}')
print(f'   F1 Score:  {metrics_att["f1"]:.4f}')
print(f'   F2 Score:  {metrics_att["f2"]:.4f} {"⭐ EXCELLENT!" if metrics_att["f2"] >= 0.70 else ""}')
print(f'   Accuracy:  {metrics_att["accuracy"]:.4f}')
print(f'   PR-AUC:    {metrics_att["pr_auc"]:.4f}')
print(f'   Confusion Matrix: TP={metrics_att["tp"]}, FP={metrics_att["fp"]}, TN={metrics_att["tn"]}, FN={metrics_att["fn"]}')

results['KAN_Attention'] = {'metrics': metrics_att, 'threshold': threshold_att}

## 📈 Step 8: Compare Results & Export

In [ ]:
# Compile and display final results
print('\n' + '='*70)
print('📊 FINAL RESULTS - CM1')
print('='*70 + '\n')

results_list = []
for model_name, data in results.items():
    m = data['metrics']
    results_list.append({
        'dataset': 'CM1',
        'model': model_name,
        'recall': m['recall'],
        'precision': m['precision'],
        'f1': m['f1'],
        'f2': m['f2'],
        'accuracy': m['accuracy'],
        'pr_auc': m['pr_auc'],
        'threshold': data['threshold']
    })

results_df = pd.DataFrame(results_list)

# Display results
print('\n📋 Model Comparison:\n')
for _, row in results_df.iterrows():
    print(f"{row['model']}:")
    print(f"   Recall:    {row['recall']:.4f} {'🎯' if row['recall'] >= 0.80 else ''}")
    print(f"   Precision: {row['precision']:.4f}")
    print(f"   F2 Score:  {row['f2']:.4f} {'⭐' if row['f2'] >= 0.75 else ''}")
    print(f"   Accuracy:  {row['accuracy']:.4f}")
    print(f"   Threshold: {row['threshold']:.2f}\n")

# Export results
csv_path = os.path.join(OUTPUT_DIR, f'results_{RUN_ID}.csv')
json_path = os.path.join(OUTPUT_DIR, f'results_{RUN_ID}.json')

results_df.to_csv(csv_path, index=False)
results_df.to_json(json_path, orient='records', indent=2)

print('\n💾 Results saved to:')
print(f'   CSV:  {csv_path}')
print(f'   JSON: {json_path}')

print('\n' + '='*70)
print('✅ EXPERIMENT COMPLETE - CM1')
print('='*70 + '\n')

# Display summary DataFrame
print('\n📊 Summary Table:')
display(results_df[['model', 'recall', 'precision', 'f1', 'f2', 'accuracy', 'pr_auc']])
